In [62]:
import pandas as pd
import numpy as np
import json

def filterOccupations(df, descriptionColumn, SOCColumn, medianColumn, meanColumn):
    """
    US wage data is cute. I love her. Didn't even need to open them up on excel to delete headers and footers because I'm too lazy to select ranges in python :) 
    Each row is tagged with an SOC group value (broad, major, minor, detailed)
    I'm going to filter out broad categories.
    If there are duplicate titles, I'm going to select the finest SOC group (major > minor > detailed).
    Then I return a simplified dataframe with the following contents

    Description   |   Median  |   Mean    |
    ---------------------------------------
    JobA          |    wage   |   wage    |
    JobB          |    wage   |   wage    |
    ...           |    ...    |   ...     |
    JobZ          |    wage   |   wage    |

    """    
    # filter out commas
    df = df.replace(',','', regex = True)
    df = df.replace('*',np.NaN)
    df = df.replace('#',np.NaN)

    # filter for optimal granularity and remove duplicates
    df = df[df[SOCColumn] != 'broad'].reset_index(drop=True)
    df['duplicates'] = df.duplicated(subset=descriptionColumn, keep='last') # because the data sets go through SOC groups with increasing granularity, we only want to keep the last duplicate if present
    df = df[~df['duplicates']].reset_index(drop=True) # removes all duplicates that aren't of the finest SOC group

    # convert to ideal datatypes
    df[medianColumn] = round(df[medianColumn].astype('float64'),2)
    df[meanColumn] = round(df[meanColumn].astype('float64'),2)

    # filter out any rows where median & mean are null
    df = df.dropna(axis=0, how='all', subset=[medianColumn,meanColumn]).reset_index(drop=True)

    # select only columns of interest
    df = df[[descriptionColumn, medianColumn, meanColumn]] 

    # rename columns for consistency with other data sets
    df.columns = ['Description', 'Median', 'Mean']
    return df

def jsonifyOccupations(df, yearInput):
    """
    Given a dataframe of occupations with mean | median salary data for a given year, write a dictionary string with the following format:
    Yeah, there's probably a way to do this with a pandas grouping or pivot but I hate pandas.
    {
        'jobA': {
            'year_1': {
                'median': 0000000,
                'mean': 0000000
            }
        },
        
        'jobB': {
            'year_1': {
                'median': 0000000,
                'mean': 0000000
            }
        },
        ...
        
        'jobZ': {
            'year_1': {
                'median': 0000000,
                'mean': 0000000
            }
        }
        
    }
    """
    d = dict.fromkeys(df.loc[:,'Description'])

    for i in range(len(df)):
        d[df.loc[i,'Description']] = {}
        content = dict.fromkeys(['median', 'mean'])
        content['median'] = df.loc[i,'Median']
        content['mean'] = df.loc[i,'Mean']
        d[df.loc[i,'Description']][str(yearInput)] = content
    return d

def merge_dictionaries(dictA, dictB):
    """
    Bruh does this function already exist wtf. This sounds like a homework problem but anyway
    Take dictB. 
    If a key in dictB is present in dictA, append the contents of the key to dictA[key] (technically, i'm creating a new key with the year from dict B and assigning it's items).
    Otherwise, add dictB[key] and it's contents as a new value in dictA.
    """
    for key in dictB:
        if key in dictA.keys():
            dictA[key][list(dictB[key].keys())[0]] = list(dictB[key].values())[0]
        else:
            dictA[key] = dictB[key]
    return dictA

US2017 = jsonifyOccupations(filterOccupations(pd.read_csv('raw data/US 2017.csv'),'OCC_TITLE', 'OCC_GROUP', 'A_MEDIAN','A_MEAN'), 2017)
US2018 = jsonifyOccupations(filterOccupations(pd.read_csv('raw data/US 2018.csv'),'OCC_TITLE', 'OCC_GROUP', 'A_MEDIAN','A_MEAN'), 2018)
US2019 = jsonifyOccupations(filterOccupations(pd.read_csv('raw data/US 2019.csv'),'occ_title', 'o_group', 'a_median','a_mean'), 2019)
US2020 = jsonifyOccupations(filterOccupations(pd.read_csv('raw data/US 2020.csv'),'OCC_TITLE', 'O_GROUP', 'A_MEDIAN','A_MEAN'), 2020)
US2021 = jsonifyOccupations(filterOccupations(pd.read_csv('raw data/US 2021.csv'),'OCC_TITLE', 'O_GROUP', 'A_MEDIAN','A_MEAN'), 2021)

temp = merge_dictionaries(US2017, US2018)
temp = merge_dictionaries(temp, US2019)
temp = merge_dictionaries(temp, US2020)
temp = merge_dictionaries(temp, US2021)

with open('processed data/US_2017-2021.json', 'w', encoding='utf-8') as f:
    json.dump(temp, f, ensure_ascii=False, indent=4)
